# Prueba Tecnica

En este notebook, en un primer momento solo se evaluara que los resultados sean correctos. Luego, se revisará temas de optimizacion de memoria y eficiencia.

In [1]:
test_file_path = "../test/resources/twitter_test.json"
file_path = "../farmers-protest-tweets-2021-2-4.json"

In [2]:
from q1_memory import q1_memory

exam = q1_memory(test_file_path)
exam

[(datetime.date(2021, 2, 24), 'anmoldhaliwal')]

In [5]:
exam = q1_memory(file_path)
exam

[(datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 14), 'rebelpacifist')]